# Script Goals:

1. Fuzzy match official county names to internal company geodatabase county names 
    (Internal county names have inconsistent spellings, have 2 counties for 1 row, etc.)
    
    
## Script Overview
1.  Pull lease records from database
2.  Read in us counties, state, fips data 
3.  Clean and join fips, official county names to company lease records through fuzzy match

In [1]:
import pandas as pd
import geopandas as gp
import datetime as dt
import pdb
import numpy as np
import requests as req

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Read in Source Data

### Connect to Company Database to Load Lease Data

In [2]:
import pymysql
from db_config import mysql_config

In [3]:
db = pymysql.connect(host = mysql_config['host'], user = mysql_config['user'], password = mysql_config['password'], database = mysql_config['database'])

In [4]:
cursor = db.cursor()
cursor.execute('select * from details')

2175

In [5]:
#pull in details table
sql = 'SELECT * FROM details;'

leases = pd.read_sql(sql, db)

### Us Counties Shapefile

In [6]:
usCounties = gp.read_file('zip://Source Files/UScounties/UScounties.zip')

usCounties.head()
usCounties.info()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry
0,Lake of the Woods,Minnesota,27,077,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715..."
1,Ferry,Washington,53,019,53019,"POLYGON ((-118.85163 47.94956, -118.84846 48.4..."
2,Stevens,Washington,53,065,53065,"POLYGON ((-117.43883 48.04412, -117.54219 48.0..."
3,Okanogan,Washington,53,047,53047,"POLYGON ((-118.97209 47.93915, -118.97406 47.9..."
4,Pend Oreille,Washington,53,051,53051,"POLYGON ((-117.43858 48.99992, -117.03205 48.9..."


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3141 entries, 0 to 3140
Data columns (total 6 columns):
NAME          3141 non-null object
STATE_NAME    3141 non-null object
STATE_FIPS    3141 non-null object
CNTY_FIPS     3141 non-null object
FIPS          3141 non-null object
geometry      3141 non-null geometry
dtypes: geometry(1), object(5)
memory usage: 147.3+ KB


In [7]:
#converting fips fields to integers for joining with state table

for col in list(filter(lambda x: "FIPS" in x, usCounties.columns)):
    print(col)
    usCounties[col] = usCounties[col].astype(int)

STATE_FIPS
CNTY_FIPS
FIPS


### State Webscrape Table

In [8]:
stlink = 'https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696'
stTable = pd.read_html(req.get(stlink).content)[0]

In [9]:

#last row had null fips due to extra read row from html source
stTable = stTable.iloc[:-1, :]
stTable["FIPS"] = stTable["FIPS"].astype(int)
stTable.rename(columns = {"FIPS": "State Fips"}, inplace = True)
stTable.head()

,Name,Postal Code,State Fips
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6


## Join US Counties Shapefile with State Table

In [10]:
usCounties = pd.merge(usCounties, stTable.iloc[:,1:], how="left", left_on = "STATE_FIPS", right_on = "State Fips")
usCounties.drop(columns ="State Fips", inplace = True)
usCounties

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry,Postal Code
0,Lake of the Woods,Minnesota,27,77,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715...",MN
1,Ferry,Washington,53,19,53019,"POLYGON ((-118.85163 47.94956, -118.84846 48.4...",WA
2,Stevens,Washington,53,65,53065,"POLYGON ((-117.43883 48.04412, -117.54219 48.0...",WA
3,Okanogan,Washington,53,47,53047,"POLYGON ((-118.97209 47.93915, -118.97406 47.9...",WA
4,Pend Oreille,Washington,53,51,53051,"POLYGON ((-117.43858 48.99992, -117.03205 48.9...",WA
5,Boundary,Idaho,16,21,16021,"POLYGON ((-117.02911 48.83808, -117.03205 48.9...",ID
6,Lincoln,Montana,30,53,30053,"POLYGON ((-116.05550 48.20848, -116.05669 48.4...",MT
7,Flathead,Montana,30,29,30029,"POLYGON ((-113.47363 47.59758, -113.63732 47.6...",MT
8,Glacier,Montana,30,35,30035,"POLYGON ((-112.18273 48.47117, -112.22915 48.4...",MT
9,Toole,Montana,30,101,30101,"POLYGON ((-111.42231 48.21776, -111.67355 48.2...",MT


# Prep and Manipulate Data for Fuzzy Match 

### Add Columns for Fuzzy Match Values (County + State)

In [11]:
usCounties["County Fuzzy Choices"] = usCounties["NAME"] + ", " + usCounties["Postal Code"]

usCounties.head()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry,Postal Code,County Fuzzy Choices
0,Lake of the Woods,Minnesota,27,77,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715...",MN,"Lake of the Woods, MN"
1,Ferry,Washington,53,19,53019,"POLYGON ((-118.85163 47.94956, -118.84846 48.4...",WA,"Ferry, WA"
2,Stevens,Washington,53,65,53065,"POLYGON ((-117.43883 48.04412, -117.54219 48.0...",WA,"Stevens, WA"
3,Okanogan,Washington,53,47,53047,"POLYGON ((-118.97209 47.93915, -118.97406 47.9...",WA,"Okanogan, WA"
4,Pend Oreille,Washington,53,51,53051,"POLYGON ((-117.43858 48.99992, -117.03205 48.9...",WA,"Pend Oreille, WA"


In [12]:
leases["County for Matching"] = leases["COUNTY"] + ", " + leases["ST"]

pd.options.display.max_rows=300
leases["County for Matching"].value_counts().sort_index()

ADAMS, MS                               9
ALCONA, MI                              6
ALFALFA, OK                             1
ALLEGAN, MI                            16
AMITE, MS                              19
AVOYELLES, LA                           5
BEAUREGARD, LA                          1
BEAVER, OK                              4
BIBB, AL                                5
BIENVILLE PARISH, LA                    1
BILLINGS, ND                            4
BLAINE, MT                              3
BLOUNT, AL                              4
BOSSIER PARISH, LA                      2
BOTTINEAU, ND                           1
BOWMAN, ND                             15
BRADFORD, PA                            5
BURLESON, TX                            3
Burleson, TX                            1
CADDO & DESOTO, LA                      1
CADDO , OK                              1
CADDO PARISH , LA                       1
CADDO PARISH, LA                        5
CALCASIEU PARISH, LA              

## Fuzzy Matching County Names

In [13]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [14]:
def fuzzyMatchCustom(query, choices, User_score_cutoff = 88):
    '''
    This function will fuzzy matching to find matches for lease serial numbers in our master leases to BLM lR2000 
    database serial numbers or counties with official county names
    Default cut off score is high to avoid false matches
    '''
    
    
    #match will hold object returned from fuzzywuzzy process library
    match = process.extractOne(query, choices, score_cutoff=User_score_cutoff)
    if type(match) == type(None):
        return "No Match"
    else:
        return match[0]

### Running FuzzyMatch on all lease records

Optimizing number of counties to parse through by filtering to relevant states

In [15]:
%%time

#initializing blank string column for fuzzy matched counties
leases['Fuzzy Matched County'] = ''

for i in range(0,len(leases)):
    
    #looping through each dataframe row 
    #filtering uscounties shapefile to the state equal to the lease's dataframe's state to optimize choices
    optimized_choice = usCounties[usCounties['Postal Code'] == leases.iloc[i]['ST']]['County Fuzzy Choices']
    
    #query will be the concatenated county, st string to compare against official us counties
    query = leases['County for Matching'].fillna('').iloc[i]
    
    #save the fuzzy matched county in variable
    fuzzymatch = fuzzyMatchCustom(query,optimized_choice, User_score_cutoff=88)           
               
    #replacing blank string value in fuzzy matched county with fuzzy matched return value
    leases.at[i,'Fuzzy Matched County'] = fuzzymatch

CPU times: user 15.7 s, sys: 135 ms, total: 15.8 s
Wall time: 17.1 s


### Exporting Full leases to Output List

In [16]:

leases.to_excel("Output Lists/FuzzyMatch Export/Initial Run on BLM Master - Fuzzy Match Counties.xlsx", index = False)


# leases=pd.read_excel("Output Lists/FuzzyMatch Export/Initial Run on BLM Master - Fuzzy Match Counties.xlsx")

## Analyzing Records that Did not Have County Match on Active Lease Dataset

In [17]:
leases["EXPIRATION_DATE"] = pd.to_datetime(leases["EXPIRATION_DATE"])

In [18]:
try:
    leases[leases["EXPIRATION_DATE"]>'2021-01-01']["Fuzzy Matched County"].value_counts().sort_index()

    noMatchesActiveLeases = leases[(leases["EXPIRATION_DATE"]>'2021-01-01') & (leases["Fuzzy Matched County"] == "No Match")]

    noMatchesActiveLeases
except:
    pass

Adams, MS               5
Alcona, MI              5
Alfalfa, OK             1
Allegan, MI            16
Amite, MS              18
Avoyelles, LA           5
Bibb, AL                3
Billings, ND            4
Blaine, MT              3
Bottineau, ND           1
Bowman, ND             15
Bradford, PA            5
Burleson, TX            2
Campbell, WY           29
Carbon, MT              9
Carbon, WY              1
Chaves, NM             16
Chouteau, MT            1
Clarke, AL              1
Cleburne, AR           11
Conecuh, AL             1
Converse, WY           17
Covington, AL          16
Covington, MS           5
Crook, WY               1
Daniels, MT            13
Dawson, MT             13
Dewey, OK               1
Eddy, NM               23
Ellis, OK               2
Emery, UT               1
Escambia, AL            6
Eureka, NV              2
Fall River, SD         24
Fallon, MT              6
Fannin, TX              3
Faulkner, AR            2
Forrest, MS            10
Franklin, AR

,LEASE_NO,PLOTTED,GRANTEE,SALE_DATE,EFFECTIVE_DATE_OF_LEASE,EXPIRATION_DATE,ACRES,COUNTY,ST,STATUS_COMMENTS,EXPIRATION_YEAR,TOWNSHIP,RANGE_PLSS,DESCRIPTION_I,DESCRIPTION_II,DESCRIPTION_III,County for Matching,Fuzzy Matched County
252,LAES056818,YES,R&R Royalty,2011-06-16,2011-08-01,2021-08-01,39.55,WINN PARISH,LA,None,2021.0,10N,1W,"MERIDIAN LOUISIANA, T10 N R1W",SEC.19 NESW,None,"WINN PARISH, LA",No Match
253,LAES056819,YES,R&R Royalty,2011-06-16,2011-08-01,2021-08-01,377.03,WINN PARISH,LA,None,2021.0,10N,1W,"MERIDIAN LOUISIANA, T10 N R1W","SEC.19 S2NE, N2NW, SWNW, N2SENW, NWSE; SEC.20,...",None,"WINN PARISH, LA",No Match
254,LAES056820,YES,R&R Royalty,2011-06-16,2011-08-01,2021-08-01,757.41,WINN PARISH,LA,None,2021.0,10N,1W,"MERIDIAN LOUISIANA, T10 N R1W","SEC.19, NWSW, S2S2, NESE, SEC.20, E2N2, E2NW, ...",None,"WINN PARISH, LA",No Match
255,LAES056821,YES,R&R Royalty,2011-06-16,2011-08-01,2021-08-01,715.99,WINN PARISH,LA,None,2021.0,10N,1W,"MERIDIAN LOUISIANA, T10 N R1W","SEC.28, NENE, SE, SEC.32, N2, N2SW, NWSE, S2SE...",None,"WINN PARISH, LA",No Match
256,LAES056822,YES,R&R Royalty,2011-06-16,2011-08-01,2021-08-01,1613.27,WINN PARISH,LA,None,2021.0,10N,1W,"MERIDIAN LOUISIANA, T10 N R1W","SEC.28, N2NW, SENW, S2NWSW, S2SW, SEC.29, NE. ...",None,"WINN PARISH, LA",No Match
257,LAES056823,YES,R&R Royalty,2011-06-16,2011-08-01,2021-08-01,242.37,WINN PARISH,LA,None,2021.0,10N,1W,"MERIDIAN LOUISIANA, T10 N R1W","SEC.29, SWNW, SEC.30, N2NE, N2SENE, SWNW, TRAC...",None,"WINN PARISH, LA",No Match
258,LAES056828,YES,R&R Royalty,2011-06-16,2011-08-01,2021-08-01,79.64,WINN PARISH,LA,None,2021.0,10N,3W,"MERIDIAN LOUISIANA, T10 N R3W","SEC.9 NENE, SEC.10, NWNW",None,"WINN PARISH, LA",No Match
259,LAES056829,YES,R&R Royalty,2011-06-16,2011-08-01,2021-08-01,79.66,WINN PARISH,LA,None,2021.0,10N,3W,"MERIDIAN LOUISIANA, T10 N R3W","SEC.22, E2SE",None,"WINN PARISH, LA",No Match
260,LAES056831,YES,R&R Royalty,2011-06-16,2011-08-01,2021-08-01,279.78,WINN PARISH,LA,None,2021.0,10N,3W,"MERIDIAN LOUISIANA, T10 N R3W","SEC.26, S2NW, SESW, SE",None,"WINN PARISH, LA",No Match
261,LAES056832,YES,R&R Royalty,2011-06-16,2011-08-01,2021-08-01,79.69,WINN PARISH,LA,None,2021.0,10N,3W,"MERIDIAN LOUISIANA, T10 N R3W",SEC.34 E2NW,None,"WINN PARISH, LA",No Match


### Noticed The Word "Parish" causes no match to be found

In [19]:
try:
    testString = noMatchesActiveLeases.iloc[0]["County for Matching"]

    process.extract(testString, usCounties["County Fuzzy Choices"])

    process.extract(testString.replace("PARISH", ""), usCounties["County Fuzzy Choices"])
except:
    pass

[('La Paz, AZ', 86, 2240),
 ('Caddo, LA', 86, 2448),
 ('Union, LA', 86, 2454),
 ('Tensas, LA', 86, 2572),
 ('Winn, LA', 86, 2584)]

[('Winn, LA', 95, 2584),
 ('West Carroll, LA', 86, 2458),
 ('East Carroll, LA', 86, 2459),
 ('Natchitoches, LA', 86, 2585),
 ('Pointe Coupee, LA', 86, 2733)]

## Rerunning Fuzzy Match After Stripping Parish From County Names

##### Stripping words: Parishes, counties, county, parish

In [20]:
wordstoreplace = ['PARISHES', 'PARISH', 'COUNTIES', 'COUNTY']

In [21]:
for word in wordstoreplace:
    print(word)
    leases["County for Matching"] = leases["County for Matching"].apply(lambda x: str(x).upper().replace(word,""))
    
leases[leases['ST']=='LA']

PARISHES
PARISH
COUNTIES
COUNTY


,LEASE_NO,PLOTTED,GRANTEE,SALE_DATE,EFFECTIVE_DATE_OF_LEASE,EXPIRATION_DATE,ACRES,COUNTY,ST,STATUS_COMMENTS,EXPIRATION_YEAR,TOWNSHIP,RANGE_PLSS,DESCRIPTION_I,DESCRIPTION_II,DESCRIPTION_III,County for Matching,Fuzzy Matched County
240,LAES050702,NO,R&R Royalty,2000-06-29,2000-10-01,2010-10-01,80.00,ST BERNARD,LA,Expired 2010,2010.0,None,None,NONE,"T14S, R16E, ST HELENA MERIDIAN","SEC 13, N2SW","ST BERNARD, LA","St. Bernard, LA"
241,LAES056361,YES,R&R Royalty,2010-06-23,2010-09-01,2020-09-01,40.00,GRANT PARISH,LA,None,2020.0,8N,1W,"MERIDIAN LA , T8N, R1W","SEC.28, SESE",None,"GRANT , LA",No Match
242,LAES056362,YES,R&R Royalty,2010-06-23,2010-09-01,2020-09-01,39.61,WINN PARISH,LA,None,2020.0,12N,5W,"MERIDIAN LA , T12N, R5W","SEC.28, NESE",None,"WINN , LA",No Match
243,LAES056363,YES,R&R Royalty,2010-06-23,2010-09-01,2020-09-01,239.92,WINN PARISH,LA,None,2020.0,12N,5W,"MERIDIAN LA, T12N , R5W","SEC.29, SW, S2NW",None,"WINN , LA",No Match
244,LAES056364,YES,R&R Royalty,2010-06-23,2010-09-01,2020-09-01,510.67,WINN PARISH,LA,None,2020.0,12N,5W,"MERIDIAN LA, T12N , R5W","SEC.35, NWNE, S2NE, S2, E2NW",None,"WINN , LA",No Match
245,LAES056365,YES,R&R Royalty,2010-06-23,2010-09-01,2020-09-01,40.08,WINN PARISH,LA,None,2020.0,12N,6W,"MERIDIAN LA, T12N , R6W","SEC. 3, SESE",None,"WINN , LA",No Match
246,LAES056366,YES,R&R Royalty,2010-06-23,2010-09-01,2020-09-01,30.44,WINN PARISH,LA,None,2020.0,12N,6W,"MERIDIAN LA, T12N, R6W","SEC. 13, W2E2NENW, W2NENW",None,"WINN , LA",No Match
247,LAES056367,YES,R&R Royalty,2010-06-23,2010-09-01,2020-09-01,634.84,WINN PARISH,LA,None,2020.0,12N,6W,"MERIDIAN LA, T12N, R6W","SEC.23, ALL LESS & EXCEPT 12.40 AC. IN RR ROW ...",None,"WINN , LA",No Match
248,LAES056369,YES,R&R Royalty,2010-06-23,2010-09-01,2020-09-01,99.41,WINN PARISH,LA,"Transfer of Operating Rights to AEEC II, LLC –...",2020.0,13N,5W,"MERIDIAN LA, T13N, R5W","SEC.5, SWNW, NWSW, N2SWSW",None,"WINN , LA",No Match
249,LAES056371,YES,R&R Royalty,2010-06-23,2010-09-01,2020-09-01,19.90,WINN PARISH,LA,None,2020.0,13N,5W,"MERIDIAN LA, T13N, R5W","SEC.13, S2NENE",None,"WINN , LA",No Match


### Re-running Fuzzy Match after cleaning/replacing words that cause mismatches

In [22]:


# leases["Fuzzy Matched County"] = leases["County for Matching"].apply(lambda x: fuzzyMatchCustom(str(x), usCounties[usCounties['Postal Code'].isin(leases['ST'].unique())]['County Fuzzy Choices'], User_score_cutoff = 88))



#### Repeating code because creating function is not useful since it is such a specific application.  Only had to run block of code 2x

In [23]:
%%time
for i in range(0,len(leases)):
    
    #looping through each dataframe row 
    #filtering uscounties shapefile to the state equal to the lease's dataframe's state to optimize choices
    optimized_choice = usCounties[usCounties['Postal Code'] == leases.iloc[i]['ST']]['County Fuzzy Choices']
    
    #query will be the concatenated county, st string to compare against official us counties
    query = leases['County for Matching'].fillna('').iloc[i]
    
    #save the fuzzy matched county in variable
    fuzzymatch = fuzzyMatchCustom(query,optimized_choice, User_score_cutoff=88)           
               
    #replacing blank string value in fuzzy matched county with fuzzy matched return value
    leases.at[i,'Fuzzy Matched County'] = fuzzymatch

CPU times: user 14.8 s, sys: 99.4 ms, total: 14.9 s
Wall time: 15.8 s


In [24]:
#checking no match counties again
leases[leases["Fuzzy Matched County"]=="No Match"]

,LEASE_NO,PLOTTED,GRANTEE,SALE_DATE,EFFECTIVE_DATE_OF_LEASE,EXPIRATION_DATE,ACRES,COUNTY,ST,STATUS_COMMENTS,EXPIRATION_YEAR,TOWNSHIP,RANGE_PLSS,DESCRIPTION_I,DESCRIPTION_II,DESCRIPTION_III,County for Matching,Fuzzy Matched County
36,ALES56208,NO,R&R Royalty,2010-03-18,2010-06-01,2020-06-01,154.30,None,AL,SALE CANCELLED - $542.50 T/B REFUNDED PER LTR ...,2020.0,None,None,None,None,None,NAN,No Match
103,ARES50949,NO,R&R Royalty,2001-03-23,None,NaT,NaN,None,None,Bid Refunded 4-2-2002,NaN,None,None,None,None,None,NAN,No Match
107,ARES50954,NO,R&R Royalty,2001-03-23,None,NaT,NaN,None,None,Bid Refunded 1-9-2002,NaN,None,None,None,None,None,NAN,No Match
109,ARES53220,NO,R&R Royalty,2008-04-24,None,NaT,NaN,None,None,R&R WAS SUCCESSFUL BIDDER AT 4/24/2008 EASTERN...,NaN,None,None,None,None,None,NAN,No Match
137,ARES55231,NO,R&R Royalty,2008-04-24,None,NaT,NaN,None,None,SALE FOR THE LEASES LISTED AND HIGHLIGHTED IN ...,NaN,None,None,None,None,None,NAN,No Match
138,ARES55232,NO,R&R Royalty,2008-04-24,None,NaT,NaN,None,None,PAYMENT FOR THE BALANCE DUE WAS NOT MADE TIMEL...,NaN,None,None,None,None,None,NAN,No Match
326,LAES51351,NO,R&R Royalty,2002-03-28,2002-05-01,2012-05-01,273.21,CADDO & DESOTO,LA,Expired 2012 - Part of Ha Ra Suh - OPR Encore/...,2012.0,None,None,WALLACE LAKE,SEE NOTICE,"SIX (6) TRACTS, SEE NOTICE","CADDO & DESOTO, LA",No Match
330,LAES53371,YES,R&R Royalty,2005-03-24,2005-06-01,2015-06-01,102.45,NATCHOITOCHES/WINN,LA,BOUGHT FROM I MEADE HUFFORD. EXPIRED IN 2015,2015.0,None,None,None,"SEC 17, SENE; SEC 19, E2SWSW; SEC 22, SEE LSE;...",None,"NATCHOITOCHES/WINN , LA",No Match
331,LAES53372,YES,R&R Royalty,2005-03-24,2005-06-01,2015-06-01,912.84,NATCHOITOCHES/WINN,LA,BOUGHT FROM I MEADE HUFFORD. EXPIRED IN 2015,2015.0,None,None,None,"SEC 30, NE,N2SE,SESE; SEC 31, NENE,S2NE,NW,SE",None,"NATCHOITOCHES/WINN , LA",No Match
363,LAES55245,NO,R&R Royalty,2008-04-24,None,NaT,NaN,None,None,INITIAL PAYMENT WAS FOREFEITED AND NO INTEREST...,NaN,None,None,None,None,None,NAN,No Match


### Exporting Full leases to Output List

In [25]:
leases.to_excel("Output Lists/FuzzyMatch Export/Final BLM Master - Fuzzy Match Counties.xlsx", index = False)

In [26]:
# leases = pd.read_excel("Output Lists/FuzzyMatch Export/Final BLM Master - Fuzzy Match Counties.xlsx")

## Joining County FIPS with Fuzzy Matched County
--- *Fips Data will allow Tableau to create geographic visualizations

In [27]:
usCounties.head(1)
leases.head(1)

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry,Postal Code,County Fuzzy Choices
0,Lake of the Woods,Minnesota,27,77,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715...",MN,"Lake of the Woods, MN"


,LEASE_NO,PLOTTED,GRANTEE,SALE_DATE,EFFECTIVE_DATE_OF_LEASE,EXPIRATION_DATE,ACRES,COUNTY,ST,STATUS_COMMENTS,EXPIRATION_YEAR,TOWNSHIP,RANGE_PLSS,DESCRIPTION_I,DESCRIPTION_II,DESCRIPTION_III,County for Matching,Fuzzy Matched County
0,ALES51293,NO,R&R Royalty,2002-03-28,2002-05-01,2012-05-01,80.25,LAMAR,AL,Expired 2012,2012.0,None,None,SEE STIPULATIONS & LSE NOTICE,"T16S,R16W HUNTSVILLE MERIDIAN","SEC 5, N2NE","LAMAR, AL","Lamar, AL"


In [28]:
#left join to keep all lease records
leases = pd.merge(leases, usCounties[["County Fuzzy Choices", "FIPS"]], how = "left", left_on = "Fuzzy Matched County", right_on = "County Fuzzy Choices")

#dropping joined column with redubdnant data
leases.drop(columns = ['County Fuzzy Choices'], inplace = True)


# Insert FuzzyMatched and Counties to DataBase

In [29]:
official_counties = leases[['LEASE_NO', 'Fuzzy Matched County', 'FIPS']]
official_counties.head()

,LEASE_NO,Fuzzy Matched County,FIPS
0,ALES51293,"Lamar, AL",1075.0
1,ALES51294,"Lamar, AL",1075.0
2,ALES55113,"Covington, AL",1039.0
3,ALES55116,"Covington, AL",1039.0
4,ALES55117,"Covington, AL",1039.0


In [30]:
official_counties[~pd.notnull(official_counties['Fuzzy Matched County'])]

,LEASE_NO,Fuzzy Matched County,FIPS


### cleaning 
### fips to str in order to be loaded with cursor


In [31]:
official_counties['FIPS'].fillna(0, inplace = True)
official_counties['FIPS'] = official_counties['FIPS'].astype(str)
official_counties.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,LEASE_NO,Fuzzy Matched County,FIPS
0,ALES51293,"Lamar, AL",1075.0
1,ALES51294,"Lamar, AL",1075.0
2,ALES55113,"Covington, AL",1039.0
3,ALES55116,"Covington, AL",1039.0
4,ALES55117,"Covington, AL",1039.0


In [32]:
official_counties.where(pd.notnull(official_counties), None, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:8834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors, try_cast=try_cast)


In [34]:
createTablesql = '''
CREATE TABLE IF NOT EXISTS MagnumBLMLeases.official_counties(
LEASE_NO VARCHAR(15) UNIQUE,
OFFICIAL_COUNTY VARCHAR(100),
FIPS INT,
FOREIGN KEY (LEASE_NO) REFERENCES details(LEASE_NO)
);
'''

cursor = db.cursor()
cursor.execute('show tables')
cursor.fetchall()

cursor.execute(createTablesql)

cursor.execute('show tables')
cursor.fetchall()

6

(('details',),
 ('interest',),
 ('investment',),
 ('recording',),
 ('sale_info',),
 ('v_activeAcreage',))

0

7

(('details',),
 ('interest',),
 ('investment',),
 ('official_counties',),
 ('recording',),
 ('sale_info',),
 ('v_activeAcreage',))

In [35]:
insertSql = '''
INSERT INTO official_counties(LEASE_NO, OFFICIAL_COUNTY, FIPS)
VALUES (%s, %s, %s)
ON DUPLICATE KEY UPDATE LEASE_NO = %s, OFFICIAL_COUNTY = %s, FIPS = %s;
'''

### Loop To Insert Table

### insert with updating clause

In [36]:
# failedindexes = []
# errormessages = []
# for index in range(0,len(official_counties)):
    
    
#     cursor = db.cursor()
    
#     #storing row in dataframe as list
#     rowvalues = list(official_counties.iloc[index].values)
    
    
  
#     values = []
    
#     #nested for loop will cycle through the values twice on each row to generate list to insert
#     #inserting each row in dataframe twice to meet conditions of update on duplicate primary key
#     for i in range(0,2):
#         for value in rowvalues:
#             values.append(value)
    
    
#     try:
#         cursor.execute(insertSql, values)
        
#     except Exception as e:
#         print(e)
#         print("error at index: {}".format(index))
#         failedindexes.append(index)
#         errormessages.append(e)
    
#     #committing after 500 inserts
#     if index > 0 and index % 500 == 0:
#         print('committing to db: {} records inserted'.format(index))
#         db.commit()

# db.commit()


#### bulk insert

In [37]:
failedindexes = []
errormessages = []


values = []
insertSql = '''
INSERT INTO official_counties(LEASE_NO, OFFICIAL_COUNTY, FIPS)
VALUES (%s, %s, %s)
'''

for index in range(0,len(official_counties)):
    
    
    cursor = db.cursor()
    
    #storing row in dataframe as list
    rowvalues = list(official_counties.iloc[index].values)
    values.append(tuple(rowvalues))
    
    
try:
    cursor.executemany(insertSql, values)

except Exception as e:
    print(e)
    print("error at index: {}".format(index))
    failedindexes.append(index)
    errormessages.append(e)

#committing after 500 inserts
if index > 0 and index % 500 == 0:
    print('committing to db: {} records inserted'.format(index))
    db.commit()

db.commit()


2175

In [38]:
failedindexes
errormessages

[]

[]

In [39]:
official_counties[official_counties.index.isin(failedindexes)]

,LEASE_NO,Fuzzy Matched County,FIPS


In [40]:
cursor.execute('select * from official_counties')
cursor.fetchall()

2175

(('ALES51293', 'Lamar, AL', 1075),
 ('ALES51294', 'Lamar, AL', 1075),
 ('ALES55113', 'Covington, AL', 1039),
 ('ALES55116', 'Covington, AL', 1039),
 ('ALES55117', 'Covington, AL', 1039),
 ('ALES55118', 'Escambia, AL', 1053),
 ('ALES55119', 'Escambia, AL', 1053),
 ('ALES55121', 'Escambia, AL', 1053),
 ('ALES55122', 'Escambia, AL', 1053),
 ('ALES55123', 'Covington, AL', 1039),
 ('ALES55124', 'Covington, AL', 1039),
 ('ALES55125', 'Covington, AL', 1039),
 ('ALES55126', 'Covington, AL', 1039),
 ('ALES55128', 'Covington, AL', 1039),
 ('ALES55129', 'Covington, AL', 1039),
 ('ALES55130', 'Covington, AL', 1039),
 ('ALES55131', 'Covington, AL', 1039),
 ('ALES55132', 'Covington, AL', 1039),
 ('ALES55138', 'Shelby, AL', 1117),
 ('ALES55311', 'Escambia, AL', 1053),
 ('ALES55543', 'Escambia, AL', 1053),
 ('ALES55546', 'Escambia, AL', 1053),
 ('ALES55549', 'Hale, AL', 1065),
 ('ALES55550', 'Hale, AL', 1065),
 ('ALES55551', 'Perry, AL', 1105),
 ('ALES55552', 'Bibb, AL', 1007),
 ('ALES55553', 'Bibb, A

In [41]:
db.close()